In [1]:
# ! pip install --upgrade pip
# ! python.exe -m pip install --upgrade pip
# ! pip install python-pptx
# ! pip install pandas
# ! pip install openpyxl

In [1]:
%run "..\general_functions\generalFunctions.ipynb"
%run "..\general_functions\Extracting Data Functions.ipynb"

Rebuilding cache of generated files for COM support...
Checking 00020813-0000-0000-C000-000000000046x0x1x9
Could not add module (IID('{00020813-0000-0000-C000-000000000046}'), 0, 1, 9) - <class 'ImportError'>: cannot import name 'Dispatch' from partially initialized module 'win32com.client' (most likely due to a circular import) (c:\Users\BW4SA\AppData\Local\Programs\Python\Python312\Lib\site-packages\win32com\client\__init__.py)
Done.


## Data Parameters

In [2]:
f_name = os.path.join(os.path.dirname(os.getcwd()),"Edgewell US Dataset.xlsx")

client_manuf = ["Edgewell"]
client_brands = ["Billie","Hydro Silk","Intuition","Skintimate"]

decimals = 2
sign = "Before"
currency = '$'
currency = ' '+ currency if sign.lower() == 'after' else  currency + ' '
 

categories = ["Manual Shave Women"]
sectors = ["Dermaplane","Disposable","System"]
segments = ["Dermaplane","Disposable","Hybrid","Razors","Refills"]
subsegments= ["1 Blade", "2 Blade", "3 Blade", "4 Blade", "5 Blade", "6 Blade","Rem Blades"]
subcategories= ["Dry Skin","Normal Skin","Rem Types","Sensitive Skin"]
 
national = True
customareas= ""
areas = ['NATIONAL',"RETAILER"]

regions_RET  = ["Amazon", "CVS Corp", "Walmart"]
channels_RET = ["Amazon 1P", "CVS", "Walmart Div1 Corp", "Walmart Nm Corp", "Walmart Sc Corp","Amazon 3P"]
market_RET = []

regions_CHAN = []
channels_CHAN = []
market_CHAN = []
 
regions_CUST = []
channels_CUST = []
market_CUST = []

data_source = "DATA SOURCE: Trade Panel/Retailer Data | Ending August 2024"
 
years = ['2022','2023', "2024"]


ManufOrTopC ="Top Companies"
BrandOrTopB = "Top Brands"


In [3]:
f_path = Path.cwd()

excel = client.gencache.EnsureDispatch('Excel.Application')
excel.Visible = True  # False
wb = excel.Workbooks.Open(f_name)
ws=wb.Sheets([s.Name for s in wb.Sheets][0])
s_name = [s.Name for s in wb.Sheets][0]
## If changed we'll need to change the iloc's of the cleaning
pvtTable = ws.PivotTables(1)

#change report layout
pvtTable.RowAxisLayout(1)   #RowAxisLayout(1) for tabular form

#change pivot table style
#Select from Design tab, try out Medium9 or Medium3
pvtTable.TableStyle2 = "pivotStyleMedium21"
pvtTable.ClearTable()

pvtTable.TableRange2.Cut(ws.Range("A16"))

fieldsNamePosition={}
for i in range(1,pvtTable.CubeFields.Count+1):
    fieldsNamePosition[str(pvtTable.CubeFields(i))]=i

In [4]:
start_time = time.time()

## SECTORS Dataframes

In [6]:
# Dictionary of DataFrames
sectors_dfs = {}
sectors_P12M_dfs = {}

In [7]:
if len(sectors)!=0:

    row_list = ['[Products].[Sector]']
    column_list=['[Calendar].[Year]']
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Time Logic].[Time Period]']
    value_list=['[Measures].[Volume Sales]','[Measures].[Value Sales]','[Measures].[Volume Share]','[Measures].[Value Share]','[Measures].[Value Sales IYA]','[Measures].[Av Price/KG]','[Measures].[WoB %]','[Measures].[Relative Price]','[Measures].[Growth Contribution]','[Measures].[Volume Sales IYA]','[Measures].[IYA Price/KG]']

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    #Select the filter values for each filter
    pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    # Segments Dataframes For P3Y For Area = NATIONAL, REGION, CHANNEL,CUSTOM

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'

    calculate_category_data(sectors_dfs)

    # Segments Dataframes For P12M For Area = NATIONAL, REGION, CHANNEL,CUSTOM

    pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Calendar].[Year]').values())[0]).Orientation = 0  

    pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
    pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'

    calculate_category_data(sectors_P12M_dfs ) 


with open('Landscape Datasets/sectors_dfs.pickle', 'wb') as handle:
    pickle.dump(sectors_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('Landscape Datasets/sectors_P12M_dfs.pickle', 'wb') as handle:
    pickle.dump(sectors_P12M_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### Client SCOPE = Sector

In [9]:
sectors_client_dfs = {}

if len(sectors)!=0:
    row_list=['[Products].[Sector]']
    column_list=[]
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Time Logic].[Time Period]',f'[Products].[{ManufOrTopC}]',f'[Products].[{BrandOrTopB}]']
    value_list=['[Measures].[Value Share]','[Measures].[Av Price/KG]','[Measures].[Value Share DYA]']

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    #Select the filter values for each filter
    pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    # Segments Dataframes For P3Y For Area = NATIONAL, REGION, CHANNEL,CUSTOM

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Sector]'


    pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
    pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'

    pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").ClearAllFilters()
    if len(client_manuf)>1:
        pvtTable.CubeFields("[Products].[{ManufOrTopC}]").EnableMultiplePageItems = True 
        
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").VisibleItemsList =[f"[Products].[{ManufOrTopC}].&[{i}]" for i in client_manuf]
    else:
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").CurrentPageName = f"[Products].[{ManufOrTopC}].&[{client_manuf[0]}]"
        
    calculate_category_data(sectors_client_dfs,f' | {client_manuf[0]}')
    if len(client_brands)!=0:    
        calculate_retailer_channel_market_Brands(sectors_client_dfs)

with open('Landscape Datasets/sectors_client_dfs.pickle', 'wb') as handle:
    pickle.dump(sectors_client_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

## SEGMENTS Dataframes

In [10]:
# Dictionary of DataFrames
segments_dfs = {}
segments_P12M_dfs = {}

In [11]:
if len(segments)!=0:
    row_list = ['[Products].[Segment]']
    column_list=['[Calendar].[Year]']
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Time Logic].[Time Period]']
    value_list=['[Measures].[Volume Sales]','[Measures].[Value Sales]','[Measures].[Volume Share]','[Measures].[Value Share]','[Measures].[Value Sales IYA]','[Measures].[Av Price/KG]','[Measures].[WoB %]','[Measures].[Relative Price]','[Measures].[Growth Contribution]','[Measures].[Volume Sales IYA]','[Measures].[IYA Price/KG]']

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    #Select the filter values for each filter
    pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    # Segments Dataframes For P3Y For Area = NATIONAL, REGION, CHANNEL,CUSTOM

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'

    calculate_category_data(segments_dfs)

    # Segments Dataframes For P12M For Area = NATIONAL, REGION, CHANNEL,CUSTOM

    pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Calendar].[Year]').values())[0]).Orientation = 0  

    pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
    pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'

    calculate_category_data(segments_P12M_dfs ) 


with open('Landscape Datasets/segments_dfs.pickle', 'wb') as handle:
    pickle.dump(segments_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('Landscape Datasets/segments_P12M_dfs.pickle', 'wb') as handle:
    pickle.dump(segments_P12M_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)


#### Client SCOPE = Segment

In [12]:
segments_client_dfs = {}

if len(segments)!=0:
    row_list=['[Products].[Segment]']
    column_list=[]
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Time Logic].[Time Period]',f'[Products].[{ManufOrTopC}]',f'[Products].[{BrandOrTopB}]']
    value_list=['[Measures].[Value Share]','[Measures].[Av Price/KG]','[Measures].[Value Share DYA]']

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    #Select the filter values for each filter
    pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    # Segments Dataframes For P3Y For Area = NATIONAL, REGION, CHANNEL,CUSTOM

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Segment]'


    pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
    pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'

    pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").ClearAllFilters()
    if len(client_manuf)>1:
        pvtTable.CubeFields(f"[Products].[{ManufOrTopC}]").EnableMultiplePageItems = True 
        
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").VisibleItemsList =[f"[Products].[{ManufOrTopC}].&[{i}]" for i in client_manuf]
    else:
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").CurrentPageName = f"[Products].[{ManufOrTopC}].&[{client_manuf[0]}]"
        
    calculate_category_data(segments_client_dfs,f' | {client_manuf[0]}')
    
    if len(client_brands)!=0:   
        calculate_retailer_channel_market_Brands(segments_client_dfs)

with open('Landscape Datasets/segments_client_dfs.pickle', 'wb') as handle:
    pickle.dump(segments_client_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

## SUBSEGMENT Dataframes

In [13]:
# Dictionary of DataFrames
subsegments_dfs = {}
subsegments_P12M_dfs = {}

In [14]:
if len(subsegments)!=0:
    row_list = ['[Products].[SubSegment]']
    column_list=['[Calendar].[Year]']
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Time Logic].[Time Period]']
    value_list=['[Measures].[Volume Sales]','[Measures].[Value Sales]','[Measures].[Volume Share]','[Measures].[Value Share]','[Measures].[Value Sales IYA]','[Measures].[Av Price/KG]','[Measures].[WoB %]','[Measures].[Relative Price]','[Measures].[Growth Contribution]','[Measures].[Volume Sales IYA]','[Measures].[IYA Price/KG]']

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    #Select the filter values for each filter
    pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    # Segments Dataframes For P3Y For Area = NATIONAL, REGION, CHANNEL,CUSTOM

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_category_data(subsegments_dfs)

    # Segments Dataframes For P12M For Area = NATIONAL, REGION, CHANNEL,CUSTOM

    pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Calendar].[Year]').values())[0]).Orientation = 0  

    pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
    pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'

    calculate_category_data(subsegments_P12M_dfs ) 

    pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
with open('Landscape Datasets/subsegments_dfs_new.pickle', 'wb') as handle:
    pickle.dump(subsegments_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('Landscape Datasets/subsegments_P12M_dfs_new.pickle', 'wb') as handle:
    pickle.dump(subsegments_P12M_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)


#### Client SCOPE = SubSegment

In [15]:
#Segments by Client brands 
subsegments_client_dfs = {}

if len(subsegments)!=0:
    row_list = ['[Products].[SubSegment]']
    column_list=[]
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Time Logic].[Time Period]',f'[Products].[{ManufOrTopC}]',f'[Products].[{BrandOrTopB}]']
    value_list=['[Measures].[Value Share]','[Measures].[Av Price/KG]','[Measures].[Value Share DYA]']

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    #Select the filter values for each filter
    pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    # Segments Dataframes For P3Y For Area = NATIONAL, REGION, CHANNEL,CUSTOM

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubSegment]'


    pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
    pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'

    pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").ClearAllFilters()
    if len(client_manuf)>1:
        pvtTable.CubeFields(f"[Products].[{ManufOrTopC}]").EnableMultiplePageItems = True 
        
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").VisibleItemsList =[f"[Products].[{ManufOrTopC}].&[{i}]" for i in client_manuf]
    else:
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").CurrentPageName = f"[Products].[{ManufOrTopC}].&[{client_manuf[0]}]"
        
    calculate_category_data(subsegments_client_dfs,f' | {client_manuf[0]}')
    
    if len(client_brands)!=0:    
        calculate_retailer_channel_market_Brands(subsegments_client_dfs)

with open('Landscape Datasets/subsegments_client_dfs_new.pickle', 'wb') as handle:
    pickle.dump(subsegments_client_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

## SUBCATEGORY Dataframes

In [16]:
subcategories_dfs = {}
subcategories_P12M_dfs = {}


if len(subcategories)!=0:
    row_list=['[Products].[SubCategory]']
    column_list=['[Calendar].[Year]',]
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Time Logic].[Time Period]']
    value_list=['[Measures].[Volume Sales]','[Measures].[Value Sales]','[Measures].[Volume Share]','[Measures].[Value Share]','[Measures].[Value Sales IYA]','[Measures].[Av Price/KG]','[Measures].[WoB %]','[Measures].[Relative Price]','[Measures].[Growth Contribution]','[Measures].[Volume Sales IYA]','[Measures].[IYA Price/KG]']

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    #Select the filter values for each filter
    pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    # Segments Dataframes For P3Y For Area = NATIONAL, REGION, CHANNEL,CUSTOM

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
    calculate_category_data(subcategories_dfs)

    # Segments Dataframes For P12M For Area = NATIONAL, REGION, CHANNEL,CUSTOM

    pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Calendar].[Year]').values())[0]).Orientation = 0  

    pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
    pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'

    calculate_category_data(subcategories_P12M_dfs ) 
   


with open('Landscape Datasets/subcategories_dfs_new.pickle', 'wb') as handle:
    pickle.dump(subcategories_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('Landscape Datasets/subcategories_P12M_dfs_new.pickle', 'wb') as handle:
    pickle.dump(subcategories_P12M_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [17]:
subcategories_client_dfs = {}

if len(subcategories)!=0:
    row_list=['[Products].[SubCategory]']
    column_list=[]
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Time Logic].[Time Period]',f'[Products].[{ManufOrTopC}]',f'[Products].[{BrandOrTopB}]']
    value_list=['[Measures].[Value Share]','[Measures].[Av Price/KG]','[Measures].[Value Share DYA]']

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    #Select the filter values for each filter
    pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[SubCategory]'
    
    pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
    pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'


    pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").ClearAllFilters()
    if len(client_manuf)>1:
        pvtTable.CubeFields(f"[Products].[{ManufOrTopC}]").EnableMultiplePageItems = True 
        
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").VisibleItemsList =[f"[Products].[{ManufOrTopC}].&[{i}]" for i in client_manuf]
    else:
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").CurrentPageName = f"[Products].[{ManufOrTopC}].&[{client_manuf[0]}]"
        
    calculate_category_data(subcategories_client_dfs,f' | {client_manuf[0]}')
    
    if len(client_brands)!=0:    
        calculate_retailer_channel_market_Brands(subcategories_client_dfs)

with open('Landscape Datasets/subcategories_client_dfs_new.pickle', 'wb') as handle:
    pickle.dump(subcategories_client_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

## MANUFACTURER Dataframes

In [21]:
row_list=[f'[Products].[{ManufOrTopC}]'] #1
column_list=['[Calendar].[Year]'] #2
filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Market].[Region]', '[Market].[Channel]', '[Market].[Market]','[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']#3
value_list=['[Measures].[Volume Sales]','[Measures].[Value Sales]','[Measures].[Volume Share]','[Measures].[Value Share]', '[Measures].[Share DYA]' ,'[Measures].[Av Price/KG]'] #4

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

In [ ]:
manuf_dfs = {}


# Manufacturer Slides by Category for Areas = NATIONAL, RETAILER, CHANNEL, CUSTOM

pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
calculate_category_data(manuf_dfs)
   

# Manufacturer Slides by Sectors for Areas = NATIONAL, RETAILER, CHANNEL, CUSTOM

calculate_sector_segment_data(manuf_dfs,'')


with open('Landscape Datasets/manuf_dfs_new.pickle', 'wb') as handle:
    pickle.dump(manuf_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [23]:
# DataFrames by Months

manuf_P12M_dfs = {}

pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Calendar].[Year]').values())[0]).Orientation = 0 #Columns (years)
pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Time Logic].[Time Period]').values())[0]).Orientation =2  
visible_items = []
for value in ['P12M', "LY"]:
    visible_items.append(f"[Time Logic].[Time Period].&[{value}]")
pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').VisibleItemsList = visible_items


# Dataframes by months by Category for Areas = NATIONAL, RETAILER, CHANNEL, CUSTOM
pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'

calculate_category_data(manuf_P12M_dfs)
    
# Dataframes by months by Sectors, segments, subseg,subcat in scope for Areas = NATIONAL, RETAILER, CHANNEL, CUSTOM
calculate_sector_segment_data(manuf_P12M_dfs,'')

with open('Landscape Datasets/manuf_P12M_dfs_new.pickle', 'wb') as handle:
    pickle.dump(manuf_P12M_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

## BRANDS Dataframes

In [28]:
row_list=[f'[Products].[{BrandOrTopB}]']
column_list=['[Calendar].[Year]']
filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Time Logic].[Time Period]', '[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list=['[Measures].[Volume Sales]','[Measures].[Value Sales]','[Measures].[Volume Share]','[Measures].[Value Share]', '[Measures].[Share DYA]' ,'[Measures].[Av Price/KG]','[Measures].[IYA Price/KG]']


pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

In [ ]:
brands_dfs = {}


# Brands Slides by Category for Areas = NATIONAL, RETAILER, CHANNEL, CUSTOM
pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'

calculate_category_data(brands_dfs)

# Manufacturer Slides by Sectors, segment, subseg, subcat in scope for Areas = NATIONAL, RETAILER, CHANNEL, CUSTOM

calculate_sector_segment_data(brands_dfs,'')


with open('Landscape Datasets/brands_dfs_new.pickle', 'wb') as handle:
    pickle.dump(brands_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [30]:
brands_P12M_dfs = {}


# DataFrames by Months

pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Calendar].[Year]').values())[0]).Orientation = 0 
pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'

# Dataframes by months by Category for Areas = NATIONAL, RETAILER, CHANNEL, CUSTOM
pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
calculate_category_data(brands_P12M_dfs)
# Dataframes by months by Sectors, segment, subseg, subcat in scope for Areas = NATIONAL, RETAILER, CHANNEL,CUSTOM

calculate_sector_segment_data(brands_P12M_dfs,'')

with open('Landscape Datasets/brands_P12M_dfs_new.pickle', 'wb') as handle:
    pickle.dump(brands_P12M_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### Manuf&Brands Evolution 

In [8]:
manuf_evolution = {}
brands_evolution = {}

In [9]:
row_list = ['[Calendar].[Year]',f'[Products].[{ManufOrTopC}]']
column_list=[]
filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Time Logic].[Time Period]', '[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list=['[Measures].[Value Sales]','[Measures].[Price Evolution]','[Measures].[Share Evolution]']


pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

pvtTable.PivotFields('[Calendar].[Year].[Year]').ClearAllFilters()
pvtTable.CubeFields('[Calendar].[Year]').EnableMultiplePageItems = True 

In [10]:
pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'


calculate_category_data(manuf_evolution)

# Dataframes by months by Sectors, segment, subseg, subcat in scope for Areas = NATIONAL, RETAILER, CHANNEL, CUSTOM


calculate_sector_segment_data(manuf_evolution,'')

{'Manual Shave Women | National':      Unnamed: 0            Unnamed: 1  Unnamed: 2 Unnamed: 3 Unnamed: 4
 0           NaN                   NaN         NaN        NaN        NaN
 1           NaN                   NaN         NaN        NaN        NaN
 2         Scope              Category         NaN        NaN        NaN
 3      Category    Manual Shave Women         NaN        NaN        NaN
 4          Area              NATIONAL         NaN        NaN        NaN
 ..          ...                   ...         ...        ...        ...
 80          NaN            Revlon Inc     4007011   1.089693   1.286325
 81          NaN  Universal Prods Mktg      851644   0.991291   0.604025
 82          NaN   Universal Razor Ind     5354554   1.072845   1.242686
 83   2024 Total                   NaN   836513612   1.140824          1
 84  Grand Total                   NaN  3485594378   1.107857          1
 
 [85 rows x 5 columns],
 'Manual Shave Women | Amazon':      Unnamed: 0           Unnamed

In [11]:
with open('Landscape Datasets/manuf_evolution_new.pickle', 'wb') as handle:
    pickle.dump(manuf_evolution, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [34]:
row_list = ['[Calendar].[Year]',f'[Products].[{BrandOrTopB}]']
column_list=[]
filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Time Logic].[Time Period]', '[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list=['[Measures].[Value Sales]','[Measures].[Price Evolution]','[Measures].[Share Evolution]']


pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

pvtTable.PivotFields('[Calendar].[Year].[Year]').ClearAllFilters()
pvtTable.CubeFields('[Calendar].[Year]').EnableMultiplePageItems = True 
# visible_items = []
# for value in ['2020','2021', '2022','2023']:
#     visible_items.append(f"[Calendar].[Year].&[{value}]")
# pvtTable.PivotFields('[Calendar].[Year].[Year]').VisibleItemsList = visible_items


In [35]:
pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'


calculate_category_data(brands_evolution)
# Dataframes by months by Sectors, segment, subseg, subcat in scope for Areas = NATIONAL, RETAILER, CHANNEL, CUSTOM
calculate_sector_segment_data(brands_evolution,'')

{'Manual Shave Women | National':       Unnamed: 0                       Unnamed: 1  Unnamed: 2 Unnamed: 3  \
 0            NaN                              NaN         NaN        NaN   
 1            NaN                              NaN         NaN        NaN   
 2          Scope                         Category         NaN        NaN   
 3       Category               Manual Shave Women         NaN        NaN   
 4           Area                         NATIONAL         NaN        NaN   
 ..           ...                              ...         ...        ...   
 165          NaN                  Universal Razor     4422903   1.015714   
 166          NaN  Universal Razor Ind: All Others      265247   0.995463   
 167          NaN                           Xtreme       43394   0.900029   
 168   2024 Total                              NaN   836513612   1.140824   
 169  Grand Total                              NaN  3485594378   1.107857   
 
     Unnamed: 4  
 0          NaN  
 1   

In [36]:
with open('Landscape Datasets/brands_evolution_new.pickle', 'wb') as handle:
    pickle.dump(brands_evolution, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Retailers, Channels, Custom Dataframes

In [38]:
retailers_dfs = {}

if "RETAILER" in areas:
    row_list=[]
    if len(regions_RET)!=0:
        row_list.append('[Market].[Region]')
    if len(channels_RET)!=0:
        row_list.append('[Market].[Channel]')
    if len(market_RET)!=0:
        row_list.append('[Market].[Market]')        
        
    column_list=[]
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Time Logic].[Time Period]', '[Products].[Sector]', '[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
    value_list=['[Measures].[Trade WoB %]','[Measures].[Trade WoB % DYA]', '[Measures].[Channel Growth Contribution]','[Measures].[Value Sales]','[Measures].[Value Sales IYA]','[Measures].[Av Price/KG]','[Measures].[Channel Relative Price]','[Measures].[Value Share]','[Measures].[IYA Price/KG]','[Measures].[Value Share DYA]']

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
    pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'

    pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
    pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[RETAILER]'

    calculate_sec_seg_subseg_subcat_only(retailers_dfs)

with open('Landscape Datasets/retailers_dfs_new.pickle', 'wb') as handle:
    pickle.dump(retailers_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [39]:
channel_dfs = {}

if "CHANNEL" in areas:
    row_list=[]
    if len(regions_CHAN)!=0:
        row_list.append('[Market].[Region]')
    if len(channels_CHAN)!=0:
        row_list.append('[Market].[Channel]')
    if len(market_CHAN)!=0:
        row_list.append('[Market].[Market]')

    column_list=[]
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Time Logic].[Time Period]', '[Products].[Sector]', '[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
    value_list=['[Measures].[Trade WoB %]','[Measures].[Trade WoB % DYA]', '[Measures].[Channel Growth Contribution]','[Measures].[Value Sales]','[Measures].[Value Sales IYA]','[Measures].[Av Price/KG]','[Measures].[Channel Relative Price]','[Measures].[Value Share]','[Measures].[IYA Price/KG]','[Measures].[Value Share DYA]']

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
    pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'

    pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
    pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[CHANNEL]'

    calculate_sec_seg_subseg_subcat_only(channel_dfs)

with open('Landscape Datasets/channel_dfs_new.pickle', 'wb') as handle:
    pickle.dump(channel_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [40]:
cust_dfs={}

if customareas in areas:
    row_list=[]
    if len(regions_CUST)!=0:
        row_list.append('[Market].[Region]')
    if len(channels_CUST)!=0:
        row_list.append('[Market].[Channel]')
    if len(market_CUST)!=0:
        row_list.append('[Market].[Market]')

    column_list=[]
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Time Logic].[Time Period]', '[Products].[Sector]', '[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
    value_list=['[Measures].[Trade WoB %]','[Measures].[Trade WoB % DYA]', '[Measures].[Channel Growth Contribution]','[Measures].[Value Sales]','[Measures].[Value Sales IYA]','[Measures].[Av Price/KG]','[Measures].[Channel Relative Price]','[Measures].[Value Share]','[Measures].[IYA Price/KG]','[Measures].[Value Share DYA]']

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
    pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'

    pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
    pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[{customareas}]'

    calculate_sec_seg_subseg_subcat_only(cust_dfs)

with open('Landscape Datasets/cust_dfs_new.pickle', 'wb') as handle:
    pickle.dump(cust_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Retailers, Channels, Custom For CLIENTS Dataframes

In [42]:
retailers_client_dfs = {}

if "RETAILER" in areas:
    row_list=[]
    if len(regions_RET)!=0:
        row_list.append('[Market].[Region]')
    if len(channels_RET)!=0:
        row_list.append('[Market].[Channel]')
    if len(market_RET)!=0:
        row_list.append('[Market].[Market]')        
    column_list=[]
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Time Logic].[Time Period]', '[Products].[Sector]', '[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]',f'[Products].[{ManufOrTopC}]',f'[Products].[{BrandOrTopB}]']
    value_list=['[Measures].[Trade WoB %]','[Measures].[Trade WoB % DYA]', '[Measures].[Channel Growth Contribution]','[Measures].[Value Sales]','[Measures].[Value Sales IYA]','[Measures].[Av Price/KG]','[Measures].[Channel Relative Price]','[Measures].[Value Share]','[Measures].[IYA Price/KG]','[Measures].[Value Share DYA]']

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
    pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'

    pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
    pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[RETAILER]'

                                ###################### By Client Manufacturer ##################

    calculate_sec_seg_subseg_subcat_Manufacturer(retailers_client_dfs)

                                #########"################ By Client Brands ####################

    calculate_sec_seg_subseg_subcat_Brands(retailers_client_dfs)

    
with open('Landscape Datasets/retailers_client_dfs_new.pickle', 'wb') as handle:
    pickle.dump(retailers_client_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [43]:
channel_client_dfs = {}

if "CHANNEL" in areas:
    row_list=[]
    if len(regions_CHAN)!=0:
        row_list.append('[Market].[Region]')
    if len(channels_CHAN)!=0:
        row_list.append('[Market].[Channel]')
    if len(market_CHAN)!=0:
        row_list.append('[Market].[Market]')        
        
    column_list=[]
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Time Logic].[Time Period]', '[Products].[Sector]', '[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]',f'[Products].[{ManufOrTopC}]',f'[Products].[{BrandOrTopB}]']
    value_list=['[Measures].[Trade WoB %]','[Measures].[Trade WoB % DYA]', '[Measures].[Channel Growth Contribution]','[Measures].[Value Sales]','[Measures].[Value Sales IYA]','[Measures].[Av Price/KG]','[Measures].[Channel Relative Price]','[Measures].[Value Share]','[Measures].[IYA Price/KG]','[Measures].[Value Share DYA]']

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
    pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'

    pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
    pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[CHANNEL]'

                                ###################### By Client Manufacturer ##################

    calculate_sec_seg_subseg_subcat_Manufacturer(channel_client_dfs)

                                ######################### By Client Brands ####################

    calculate_sec_seg_subseg_subcat_Brands(channel_client_dfs)

    
with open('Landscape Datasets/channel_client_dfs_new.pickle', 'wb') as handle:
    pickle.dump(channel_client_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [44]:
cust_client_dfs={}

if customareas in areas:
    row_list=[]
    if len(regions_CUST)!=0:
        row_list.append('[Market].[Region]')
    if len(channels_CUST)!=0:
        row_list.append('[Market].[Channel]')
    if len(market_CUST)!=0:
        row_list.append('[Market].[Market]')        
        
    column_list=[]
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Time Logic].[Time Period]', '[Products].[Sector]', '[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]',f'[Products].[{ManufOrTopC}]',f'[Products].[{BrandOrTopB}]']
    value_list=['[Measures].[Trade WoB %]','[Measures].[Trade WoB % DYA]', '[Measures].[Channel Growth Contribution]','[Measures].[Value Sales]','[Measures].[Value Sales IYA]','[Measures].[Av Price/KG]','[Measures].[Channel Relative Price]','[Measures].[Value Share]','[Measures].[IYA Price/KG]','[Measures].[Value Share DYA]']

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
    pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'

    pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
    pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[{customareas}]'
                                ###################### By Client Manufacturer ##################

    calculate_sec_seg_subseg_subcat_Manufacturer(cust_client_dfs)

                                ######################### By Client Brands ####################

    calculate_sec_seg_subseg_subcat_Brands(cust_client_dfs)

    
with open('Landscape Datasets/cust_client_dfs_new.pickle', 'wb') as handle:
    pickle.dump(cust_client_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Calendar Dataframes

In [45]:
calendar_dfs = {}

In [46]:
row_list = ['[Calendar].[MonthYear]']
column_list=[]
filter_list=['[Scope].[Scope]','[Products].[Category]',f'[Products].[{BrandOrTopB}]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]', '[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list=['[Measures].[Value Sales]','[Measures].[Av Price/KG]']
pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)


In [47]:
# Dataframes by months by Category for Areas = NATIONAL, RETAILER, CHANNEL, CUSTOM

pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
calculate_category_data(calendar_dfs)


# Dataframes by months by Sectors, segments, subseg, subcat in scope for Areas = NATIONAL, RETAILER, CHANNEL, CUSTOM

calculate_sector_segment_data(calendar_dfs,'')

# Brands

pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'

 ###################################### Area = National #########################################
if national:
    pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
    pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[NATIONAL]'
    calendar_dfs = create_dataframe(client_brands,calendar_dfs,f"[Products].[{BrandOrTopB}].[{BrandOrTopB}]",pvtTable,'National | ')

###################################### Area = Retailer #########################################
if "RETAILER" in areas:        
    pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
    pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[RETAILER]'

    if len(regions_RET) != 0:
        calendar_dfs= create_dataframe(client_brands,calendar_dfs,f"[Products].[{BrandOrTopB}].[{BrandOrTopB}]",pvtTable, '',regions_RET,"[Market].[Region].[Region]")
        pvtTable.PivotFields("[Market].[Region].[Region]").ClearAllFilters()
    if len(market_RET) != 0:
        calendar_dfs = create_dataframe(client_brands,calendar_dfs,f"[Products].[{BrandOrTopB}].[{BrandOrTopB}]",pvtTable, '',market_RET,"[Market].[Market].[Market]")
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters() 
    if len(channels_RET) != 0:
        calendar_dfs= create_dataframe(client_brands,calendar_dfs,f"[Products].[{BrandOrTopB}].[{BrandOrTopB}]",pvtTable, '',channels_RET,"[Market].[Channel].[Channel]")
        pvtTable.PivotFields("[Market].[Channel].[Channel]").ClearAllFilters()

            ###################################### Area = Channel #########################################

if "CHANNEL" in areas:
    pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
    pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[CHANNEL]'

    if len(regions_CHAN) != 0:
        calendar_dfs= create_dataframe(client_brands,calendar_dfs,f"[Products].[{BrandOrTopB}].[{BrandOrTopB}]",pvtTable, '',regions_CHAN,"[Market].[Region].[Region]")
        pvtTable.PivotFields("[Market].[Region].[Region]").ClearAllFilters()
    if len(channels_CHAN) != 0:
            calendar_dfs = create_dataframe(client_brands,calendar_dfs,f"[Products].[{BrandOrTopB}].[{BrandOrTopB}]",pvtTable, '',channels_CHAN,"[Market].[Channel].[Channel]")
            pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()
    if len(market_CHAN) != 0:
        calendar_dfs= create_dataframe(client_brands,calendar_dfs,f"[Products].[{BrandOrTopB}].[{BrandOrTopB}]",pvtTable, '',market_CHAN,"[Market].[Market].[Market]")
        pvtTable.PivotFields("[Market].[Market].[Market]").ClearAllFilters()

###################################### Area = POS #########################################
if customareas in areas:
    pvtTable.PivotFields('[Market].[Area].[Area]').ClearAllFilters()
    pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[{customareas}]'

    if len(regions_CUST) != 0:
        calendar_dfs = create_dataframe(client_brands,calendar_dfs,f"[Products].[{BrandOrTopB}].[{BrandOrTopB}]",pvtTable, '',regions_CUST,"[Market].[Region].[Region]")
        pvtTable.PivotFields('[Market].[Region].[Region]').ClearAllFilters()

    if len(channels_CUST) != 0:
        calendar_dfs = create_dataframe(client_brands,calendar_dfs,f"[Products].[{BrandOrTopB}].[{BrandOrTopB}]",pvtTable, '',channels_CUST,"[Market].[Channel].[Channel]")
        pvtTable.PivotFields('[Market].[Channel].[Channel]').ClearAllFilters()

    if len(market_CUST) != 0:
        calendar_dfs = create_dataframe(client_brands,calendar_dfs,f"[Products].[{BrandOrTopB}].[{BrandOrTopB}]",pvtTable, '',market_CUST,"[Market].[Market].[Market]")
        pvtTable.PivotFields('[Market].[Market].[Market]').ClearAllFilters()


In [48]:
with open('Landscape Datasets/calendar_dfs_new.pickle', 'wb') as handle:
    pickle.dump(calendar_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Category Overview Dataframes

In [49]:
categories_overview_dfs = {}
categories_overview_manuf_dfs = {}

In [50]:
row_list = []
column_list=[]
filter_list=['[Products].[Category]',"[Scope].[Scope]",f'[Products].[{ManufOrTopC}]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]', '[Time Logic].[Time Period]']
value_list=['[Measures].[Volume Sales]','[Measures].[Value Sales]','[Measures].[Volume Sales IYA]','[Measures].[Value Sales IYA]','[Measures].[IYA Price/KG]']

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'

pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

In [51]:
pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
calculate_category_data(categories_overview_dfs)

############################################# MANUFACTURER #####################################################
pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").ClearAllFilters()
pvtTable.PivotFields(f'[Products].[{ManufOrTopC}].[{ManufOrTopC}]').CurrentPageName = f'[Products].[{ManufOrTopC}].&[{client_manuf[0]}]'

calculate_category_data(categories_overview_manuf_dfs)


{'Manual Shave Women | National':        Unnamed: 0          Unnamed: 1        Unnamed: 2       Unnamed: 3  \
 0             NaN                 NaN               NaN              NaN   
 1             NaN                 NaN               NaN              NaN   
 2             NaN                 NaN               NaN              NaN   
 3             NaN                 NaN               NaN              NaN   
 4             NaN                 NaN               NaN              NaN   
 5        Category  Manual Shave Women               NaN              NaN   
 6           Scope            Category               NaN              NaN   
 7   Top Companies            Edgewell               NaN              NaN   
 8            Area            NATIONAL               NaN              NaN   
 9          Region                 All               NaN              NaN   
 10        Channel                 All               NaN              NaN   
 11         Market                 All     

In [52]:
categories_values_dfs = {}

In [53]:
row_list = ['[Calendar].[MonthYear]','[Products].[Sector]']
column_list=[]
filter_list=['[Products].[Category]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]']
value_list=['[Measures].[Value Sales]']


pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)


pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'


In [54]:
calculate_category_data(categories_values_dfs)

{'Manual Shave Women | National':        Unnamed: 0  Unnamed: 1  Unnamed: 2
 0             NaN         NaN         NaN
 1             NaN         NaN         NaN
 2             NaN         NaN         NaN
 3             NaN         NaN         NaN
 4             NaN         NaN         NaN
 ..            ...         ...         ...
 151        Aug-24  Dermaplane     7237101
 152           NaN  Disposable    43039019
 153           NaN      System    67974309
 154  Aug-24 Total         NaN   118250429
 155   Grand Total         NaN  3485594378
 
 [156 rows x 3 columns],
 'Manual Shave Women | Amazon':        Unnamed: 0  Unnamed: 1 Unnamed: 2
 0             NaN         NaN        NaN
 1             NaN         NaN        NaN
 2             NaN         NaN        NaN
 3             NaN         NaN        NaN
 4             NaN         NaN        NaN
 ..            ...         ...        ...
 116  Jul-24 Total         NaN   13437757
 117        Aug-24  Disposable    2083862
 118           

In [55]:
with open('Landscape Datasets/categories_overview_dfs_new.pickle', 'wb') as handle:
    pickle.dump(categories_overview_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('Landscape Datasets/categories_overview_manuf_dfs_new.pickle', 'wb') as handle:
    pickle.dump(categories_overview_manuf_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('Landscape Datasets/categories_values_dfs_new.pickle', 'wb') as handle:
    pickle.dump(categories_values_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Client Sec,Seg,.. SCOPE = CATEGORY

In [56]:
sectors_client_dfs_category = {}

if len(sectors)!=0:
    row_list=['[Products].[Sector]']
    column_list=[]
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Time Logic].[Time Period]',f'[Products].[{ManufOrTopC}]',f'[Products].[{BrandOrTopB}]']
    value_list=['[Measures].[Value Sales]','[Measures].[Value Share]','[Measures].[Value Sales IYA]','[Measures].[Av Price/KG]','[Measures].[WoB %]','[Measures].[Volume Sales IYA]','[Measures].[IYA Price/KG]']
    
    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    #Select the filter values for each filter
    pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    # Segments Dataframes For P3Y For Area = NATIONAL, REGION, CHANNEL,CUSTOM

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'


    pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
    pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'

    pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").ClearAllFilters()
    if len(client_manuf)>1:
        pvtTable.CubeFields(f"[Products].[{ManufOrTopC}]").EnableMultiplePageItems = True 
        
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").VisibleItemsList =[f"[Products].[{ManufOrTopC}].&[{i}]" for i in client_manuf]
    else:
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").CurrentPageName = f"[Products].[{ManufOrTopC}].&[{client_manuf[0]}]"
        
    calculate_category_data(sectors_client_dfs_category,f' | {client_manuf[0]}')
    if len(client_brands)!=0:
        calculate_retailer_channel_market_Brands(sectors_client_dfs_category)

with open('Landscape Datasets/sectors_client_dfs_category.pickle', 'wb') as handle:
    pickle.dump(sectors_client_dfs_category, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [57]:
segments_client_dfs_category = {}

if len(segments)!=0:
    row_list=['[Products].[Segment]']
    column_list=[]
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Time Logic].[Time Period]',f'[Products].[{ManufOrTopC}]',f'[Products].[{BrandOrTopB}]']
    value_list=['[Measures].[Value Sales]','[Measures].[Value Share]','[Measures].[Value Sales IYA]','[Measures].[Av Price/KG]','[Measures].[WoB %]','[Measures].[Volume Sales IYA]','[Measures].[IYA Price/KG]']
    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    #Select the filter values for each filter
    pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    # Segments Dataframes For P3Y For Area = NATIONAL, REGION, CHANNEL,CUSTOM

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'


    pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
    pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'

    pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").ClearAllFilters()
    if len(client_manuf)>1:
        pvtTable.CubeFields(f"[Products].[{ManufOrTopC}]").EnableMultiplePageItems = True 
        
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").VisibleItemsList =[f"[Products].[{ManufOrTopC}].&[{i}]" for i in client_manuf]
    else:
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").CurrentPageName = f"[Products].[{ManufOrTopC}].&[{client_manuf[0]}]"
        
    calculate_category_data(segments_client_dfs_category,f' | {client_manuf[0]}')
    
    if len(client_brands)!=0:
        calculate_retailer_channel_market_Brands(segments_client_dfs_category)

with open('Landscape Datasets/segments_client_dfs_category.pickle', 'wb') as handle:
    pickle.dump(segments_client_dfs_category, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [58]:
subsegments_client_dfs_category = {}

if len(subsegments)!=0:
    row_list=['[Products].[SubSegment]']
    column_list=[]
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Time Logic].[Time Period]',f'[Products].[{ManufOrTopC}]',f'[Products].[{BrandOrTopB}]']
    value_list=['[Measures].[Value Sales]','[Measures].[Value Share]','[Measures].[Value Sales IYA]','[Measures].[Av Price/KG]','[Measures].[WoB %]','[Measures].[Volume Sales IYA]','[Measures].[IYA Price/KG]']

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    #Select the filter values for each filter
    pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    # Segments Dataframes For P3Y For Area = NATIONAL, REGION, CHANNEL,CUSTOM

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'


    pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
    pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'

    pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").ClearAllFilters()
    if len(client_manuf)>1:
        pvtTable.CubeFields(f"[Products].[{ManufOrTopC}]").EnableMultiplePageItems = True 
        
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").VisibleItemsList =[f"[Products].[{ManufOrTopC}].&[{i}]" for i in client_manuf]
    else:
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").CurrentPageName = f"[Products].[{ManufOrTopC}].&[{client_manuf[0]}]"
        
    calculate_category_data(subsegments_client_dfs_category,f' | {client_manuf[0]}')
    if len(client_brands)!=0:
        calculate_retailer_channel_market_Brands(subsegments_client_dfs_category)

with open('Landscape Datasets/subsegments_client_dfs_category.pickle', 'wb') as handle:
    pickle.dump(subsegments_client_dfs_category, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [59]:
subcategories_client_dfs_category = {}

if len(subcategories)!=0:
    row_list=['[Products].[SubCategory]']
    column_list=[]
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Time Logic].[Time Period]',f'[Products].[{ManufOrTopC}]',f'[Products].[{BrandOrTopB}]']
    value_list=['[Measures].[Value Sales]','[Measures].[Value Share]','[Measures].[Value Sales IYA]','[Measures].[Av Price/KG]','[Measures].[WoB %]','[Measures].[Volume Sales IYA]','[Measures].[IYA Price/KG]']

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

    #Select the filter values for each filter
    pvtTable.PivotFields("[Products].[Category].[Category]").ClearAllFilters()
    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    # Segments Dataframes For P3Y For Area = NATIONAL, REGION, CHANNEL,CUSTOM

    pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
    pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'


    pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").ClearAllFilters()
    pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'

    pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").ClearAllFilters()
    if len(client_manuf)>1:
        pvtTable.CubeFields(f"[Products].[{ManufOrTopC}]").EnableMultiplePageItems = True 
        
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").VisibleItemsList =[f"[Products].[{ManufOrTopC}].&[{i}]" for i in client_manuf]
    else:
        pvtTable.PivotFields(f"[Products].[{ManufOrTopC}].[{ManufOrTopC}]").CurrentPageName = f"[Products].[{ManufOrTopC}].&[{client_manuf[0]}]"
        
    calculate_category_data(subcategories_client_dfs_category,f' | {client_manuf[0]}')
    if len(client_brands)!=0:
        
        calculate_retailer_channel_market_Brands(subcategories_client_dfs_category)

with open('Landscape Datasets/subcategories_client_dfs_category.pickle', 'wb') as handle:
    pickle.dump(subcategories_client_dfs_category, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Momentum Analysis Dataframes

In [60]:
momentum_retailers_dfs = {}

if "RETAILER" in areas:
    row_list=[f'[Products].[{BrandOrTopB}]']
    if len(regions_RET)!=0:
        row_list.append('[Market].[Region]')
    if len(channels_RET)!=0:
        row_list.append('[Market].[Channel]')
    if len(market_RET)!=0:
        row_list.append('[Market].[Market]')        
        
    column_list=['[Time Logic].[Time Period]']
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]', '[Products].[Sector]', '[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
    value_list=[]

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
    
    visible_items = []
    for value in ["P12M", "P3M"]:
        visible_items.append(f"[Time Logic].[Time Period].&[{value}]")
    pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").VisibleItemsList = visible_items 

    pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[RETAILER]'

    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    value_list=['[Measures].[Value Sales]','[Measures].[Value Share DYA]','[Measures].[Value Share]']
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)


    calculate_sec_seg_subseg_subcat_only(momentum_retailers_dfs)

with open('Landscape Datasets/momentum_retailers_dfs.pickle', 'wb') as handle:
    pickle.dump(momentum_retailers_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [61]:
momentum_channel_dfs = {}

if "CHANNEL" in areas:
    row_list=[f'[Products].[{BrandOrTopB}]']
    if len(regions_CHAN)!=0:
        row_list.append('[Market].[Region]')
    if len(channels_CHAN)!=0:
        row_list.append('[Market].[Channel]')
    if len(market_CHAN)!=0:
        row_list.append('[Market].[Market]')

    column_list=['[Time Logic].[Time Period]']
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]', '[Products].[Sector]', '[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
    value_list=[]

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
    
    visible_items = []
    for value in ["P12M", "P3M"]:
        visible_items.append(f"[Time Logic].[Time Period].&[{value}]")
    pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").VisibleItemsList = visible_items 

    pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[CHANNEL]'

    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    value_list=['[Measures].[Value Sales]','[Measures].[Value Share DYA]','[Measures].[Value Share]']
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
 

    calculate_sec_seg_subseg_subcat_only(momentum_channel_dfs)

with open('Landscape Datasets/momentum_channel_dfs.pickle', 'wb') as handle:
    pickle.dump(momentum_channel_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [62]:
momentum_cust_dfs={}

if customareas in areas:
    row_list=[f'[Products].[{BrandOrTopB}]']
    if len(regions_CUST)!=0:
        row_list.append('[Market].[Region]')
    if len(channels_CUST)!=0:
        row_list.append('[Market].[Channel]')
    if len(market_CUST)!=0:
        row_list.append('[Market].[Market]')

    column_list=['[Time Logic].[Time Period]']
    filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]', '[Products].[Sector]', '[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
    value_list=[]

    pvtTable.ClearTable()
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
    
    visible_items = []
    for value in ["P12M", "P3M"]:
        visible_items.append(f"[Time Logic].[Time Period].&[{value}]")
    pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").VisibleItemsList = visible_items 

    pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[{customareas}]'

    pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

    value_list=['[Measures].[Value Sales]','[Measures].[Value Share DYA]','[Measures].[Value Share]']
    pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
 
    calculate_sec_seg_subseg_subcat_only(momentum_cust_dfs)

with open('Landscape Datasets/momentum_cust_dfs.pickle', 'wb') as handle:
    pickle.dump(momentum_cust_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [63]:
momentum_dfs = {}

row_list=[f'[Products].[{BrandOrTopB}]', '[Products].[Sector]']
column_list=['[Time Logic].[Time Period]']
filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]']
value_list=[]

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

visible_items = []
for value in ["P12M", "P3M"]:
    visible_items.append(f"[Time Logic].[Time Period].&[{value}]")
pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").VisibleItemsList = visible_items 


pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'
value_list=['[Measures].[Value Sales]','[Measures].[Value Share DYA]','[Measures].[Value Share]']
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

calculate_category_data(momentum_dfs)


with open('Landscape Datasets/momentum_dfs.pickle', 'wb') as handle:
    pickle.dump(momentum_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [64]:
if any([segments, subsegments, subcategories]):
    if "RETAILER" in areas:
        row_list=[f'[Products].[{BrandOrTopB}]']
        if len(regions_RET)!=0:
            row_list.append('[Market].[Region]')
        if len(channels_RET)!=0:
            row_list.append('[Market].[Channel]')
        if len(market_RET)!=0:
            row_list.append('[Market].[Market]')      
        
                    
        column_list=['[Time Logic].[Time Period]']
        filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]']
        value_list=[]


        pvtTable.ClearTable()
        pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
        
        visible_items = []
        for value in ["P12M", "P3M"]:
            visible_items.append(f"[Time Logic].[Time Period].&[{value}]")
        pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").VisibleItemsList = visible_items 

        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[RETAILER]'

        pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'

        value_list=['[Measures].[Value Sales]','[Measures].[Value Share DYA]','[Measures].[Value Share]']
        pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

        if segments:
            momentum_retailers_seg_dfs = {}
            row_list.append('[Products].[Sector]')  
            row_list.append('[Products].[Segment]')        

            pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

            wb.Save()
            con = pd.read_excel(f_name, sheet_name=s_name)        
            momentum_retailers_seg_dfs[f'{categories[0]} | Retailer'] = con  

            with open('Landscape Datasets/momentum_retailers_seg_dfs.pickle', 'wb') as handle:
                pickle.dump(momentum_retailers_seg_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[Segment]').values())[0]).Orientation = 0

        if subsegments:
            momentum_retailers_subseg_dfs = {}
            if not segments:
                row_list.append('[Products].[Sector]')
            else:
                row_list.remove('[Products].[Segment]') 

            row_list.append('[Products].[SubSegment]')         

            pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

            wb.Save()
            con = pd.read_excel(f_name, sheet_name=s_name)        
            momentum_retailers_subseg_dfs[f'{categories[0]} | Retailer'] = con  

            with open('Landscape Datasets/momentum_retailers_subseg_dfs.pickle', 'wb') as handle:
                pickle.dump(momentum_retailers_subseg_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[Segment]').values())[0]).Orientation = 0 
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[SubSegment]').values())[0]).Orientation = 0 

        if subcategories:
            
            momentum_retailers_subcatg_dfs = {}
            if not segments or subsegments:
                row_list.append('[Products].[Sector]') 
            else:
                row_list.remove('[Products].[Segment]')
                row_list.remove('[Products].[SubSegment]') 

            row_list.append('[Products].[SubCategory]') 

            pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

            wb.Save()
            con = pd.read_excel(f_name, sheet_name=s_name)        
            momentum_retailers_subcatg_dfs[f'{categories[0]} | Retailer'] = con  

            with open('Landscape Datasets/momentum_retailers_subcatg_dfs.pickle', 'wb') as handle:
                pickle.dump(momentum_retailers_subcatg_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [65]:
if any([segments, subsegments, subcategories]):
    if "CHANNEL" in areas:
        row_list=[f'[Products].[{BrandOrTopB}]']
        if len(regions_CHAN)!=0:
            row_list.append('[Market].[Region]')
        if len(channels_CHAN)!=0:
            row_list.append('[Market].[Channel]')
        if len(market_CHAN)!=0:
            row_list.append('[Market].[Market]')      
        
                    
        column_list=['[Time Logic].[Time Period]']
        filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]']
        value_list=[]


        pvtTable.ClearTable()
        pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
        
        visible_items = []
        for value in ["P12M", "P3M"]:
            visible_items.append(f"[Time Logic].[Time Period].&[{value}]")
        pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").VisibleItemsList = visible_items 

        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[CHANNEL]'

        pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'

        value_list=['[Measures].[Value Sales]','[Measures].[Value Share DYA]','[Measures].[Value Share]']
        pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

        if segments:
            momentum_channels_seg_dfs = {}
            row_list.append('[Products].[Sector]')  
            row_list.append('[Products].[Segment]')        

            pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

            wb.Save()
            con = pd.read_excel(f_name, sheet_name=s_name)        
            momentum_channels_seg_dfs[f'{categories[0]} | Channel'] = con  

            with open('Landscape Datasets/momentum_channels_seg_dfs.pickle', 'wb') as handle:
                pickle.dump(momentum_channels_seg_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[Segment]').values())[0]).Orientation = 0

        if subsegments:
            
            momentum_channels_subseg_dfs = {}
            if not segments:
                row_list.append('[Products].[Sector]')
            else:
                row_list.remove('[Products].[Segment]')      
            row_list.append('[Products].[SubSegment]')         

            pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

            wb.Save()
            con = pd.read_excel(f_name, sheet_name=s_name)        
            momentum_channels_subseg_dfs[f'{categories[0]} | Channel'] = con  

            with open('Landscape Datasets/momentum_channels_subseg_dfs.pickle', 'wb') as handle:
                pickle.dump(momentum_channels_subseg_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[Segment]').values())[0]).Orientation = 0 
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[SubSegment]').values())[0]).Orientation = 0 

        if subcategories:
            
            momentum_channels_subcatg_dfs = {}
            if not segments or subsegments:
                row_list.append('[Products].[Sector]')
            else:
                row_list.remove('[Products].[Segment]')
                row_list.remove('[Products].[SubSegment]')      

            row_list.append('[Products].[SubCategory]') 

            pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

            wb.Save()
            con = pd.read_excel(f_name, sheet_name=s_name)        
            momentum_channels_subcatg_dfs[f'{categories[0]} | Channel'] = con  

            with open('Landscape Datasets/momentum_channels_subcatg_dfs.pickle', 'wb') as handle:
                pickle.dump(momentum_channels_subcatg_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [66]:
if any([segments, subsegments, subcategories]):
    if customareas in areas:
        row_list=[f'[Products].[{BrandOrTopB}]']
        if len(regions_CUST)!=0:
            row_list.append('[Market].[Region]')
        if len(channels_CUST)!=0:
            row_list.append('[Market].[Channel]')
        if len(market_CUST)!=0:
            row_list.append('[Market].[Market]')      
        
                    
        column_list=['[Time Logic].[Time Period]']
        filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]']
        value_list=[]


        pvtTable.ClearTable()
        pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)
        
        visible_items = []
        for value in ["P12M", "P3M"]:
            visible_items.append(f"[Time Logic].[Time Period].&[{value}]")
        pvtTable.PivotFields("[Time Logic].[Time Period].[Time Period]").VisibleItemsList = visible_items 

        pvtTable.PivotFields('[Market].[Area].[Area]').CurrentPageName = f'[Market].[Area].&[{customareas}]'

        pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

        pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'

        value_list=['[Measures].[Value Sales]','[Measures].[Value Share DYA]','[Measures].[Value Share]']
        pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

        if segments:
            momentum_cust_seg_dfs = {}
            row_list.append('[Products].[Sector]')  
            row_list.append('[Products].[Segment]')        

            pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

            wb.Save()
            con = pd.read_excel(f_name, sheet_name=s_name)        
            momentum_cust_seg_dfs[f'{categories[0]} | Channel'] = con  

            with open('Landscape Datasets/momentum_cust_seg_dfs.pickle', 'wb') as handle:
                pickle.dump(momentum_cust_seg_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[Segment]').values())[0]).Orientation = 0

        if subsegments:
            
            momentum_cust_subseg_dfs = {}
            if not segments:
                row_list.append('[Products].[Sector]')
            else:
                row_list.remove('[Products].[Segment]')      
            row_list.append('[Products].[SubSegment]')         

            pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

            wb.Save()
            con = pd.read_excel(f_name, sheet_name=s_name)        
            momentum_cust_subseg_dfs[f'{categories[0]} | Channel'] = con  

            with open('Landscape Datasets/momentum_cust_subseg_dfs.pickle', 'wb') as handle:
                pickle.dump(momentum_cust_subseg_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[Segment]').values())[0]).Orientation = 0 
        pvtTable.CubeFields(list(filter_dictionary_keys(fieldsNamePosition, '[Products].[SubSegment]').values())[0]).Orientation = 0 

        if subcategories:
            
            momentum_cust_subcatg_dfs = {}
            if not segments or subsegments:
                row_list.append('[Products].[Sector]')
            else:
                row_list.remove('[Products].[Segment]')
                row_list.remove('[Products].[SubSegment]') 

            row_list.append('[Products].[SubCategory]') 

            pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

            wb.Save()
            con = pd.read_excel(f_name, sheet_name=s_name)        
            momentum_cust_subcatg_dfs[f'{categories[0]} | Channel'] = con  

            with open('Landscape Datasets/momentum_cust_subcatg_dfs.pickle', 'wb') as handle:
                pickle.dump(momentum_cust_subcatg_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Revenue PVM Dataframes

In [67]:
# filter_dictionary_keys(fieldsNamePosition, 'Compa')

In [68]:
revenue_PVM_manuf = {}


row_list = [f'[Products].[{ManufOrTopC}]','[PVM].[Revenue Group]']
column_list=[]
filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Time Logic].[Time Period]','[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list=[]

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'

pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'

value_list=['[Measures].[Revenue By PVM]','[Measures].[Value Share]']
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)


calculate_category_data(revenue_PVM_manuf)


# Dataframes by months by Sectors, segments, subseg, subcat in scope for Areas = NATIONAL, RETAILER, CHANNEL, CUSTOM

calculate_sector_segment_data(revenue_PVM_manuf,'')


with open('Landscape Datasets/revenue_PVM_manuf.pickle', 'wb') as handle:
    pickle.dump(revenue_PVM_manuf, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [70]:
# revenue_PVM_brand = {}

row_list = [f'[Products].[{BrandOrTopB}]','[PVM].[Revenue Group]']
column_list=[]
filter_list=['[Scope].[Scope]','[Products].[Category]','[Market].[Area]','[Market].[Region]','[Market].[Channel]','[Market].[Market]','[Time Logic].[Time Period]','[Products].[Sector]','[Products].[Segment]','[Products].[SubSegment]','[Products].[SubCategory]']
value_list=[]

pvtTable.ClearTable()
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)

pvtTable.PivotFields('[Products].[Category].[Category]').CurrentPageName = f'[Products].[Category].&[{categories[0]}]'

pvtTable.PivotFields('[Time Logic].[Time Period].[Time Period]').CurrentPageName = '[Time Logic].[Time Period].&[P12M]'


pvtTable.PivotFields("[Scope].[Scope].[Scope]").ClearAllFilters()
pvtTable.PivotFields('[Scope].[Scope].[Scope]').CurrentPageName = '[Scope].[Scope].&[Category]'

value_list=['[Measures].[Revenue By PVM]','[Measures].[Value Share]']
pvtTable = set_excel_fields(row_list,column_list,filter_list,value_list,pvtTable)


calculate_category_data(revenue_PVM_brand)


# Dataframes by months by Sectors, segments, subseg, subcat in scope for Areas = NATIONAL, RETAILER, CHANNEL, CUSTOM

calculate_sector_segment_data(revenue_PVM_brand,'')


with open('Landscape Datasets/revenue_PVM_brand.pickle', 'wb') as handle:
    pickle.dump(revenue_PVM_brand, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [71]:
end_time = time.time()

elapsed_time = end_time - start_time
print(f"Script started at: {time.ctime(start_time)}")
print(f"Script ended at: {time.ctime(end_time)}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Script started at: Thu Dec 12 10:00:29 2024
Script ended at: Thu Dec 12 15:48:26 2024
Elapsed time: 20876.97 seconds
